#Suponga que tiene una lista de 15 alimentos con número de calorías, gramos de proteína, gramos de grasa y gramos de carbohidratos. Suponga que un individuo debe consumir determinada cantidad de calorías, proteínas, grasa y carbohidratos. Genere una dieta para ese individuo.

Nos vamos a valer de la lista proporcionada del profesor, "lista de alimentos.txt", y de ella vamos a sacar 15 alimentos al azar para generar la dieta.

Este programa implementa un Algoritmo Genético (AG) para generar una dieta óptima basada en una lista de alimentos con información nutricional. El objetivo es cumplir con ciertos requisitos específicos de calorías, proteínas, grasas y carbohidratos.


El programa comienza cargando una lista de alimentos desde un archivo externo llamado "Lista de alimentos.txt".

Cada línea del archivo representa un alimento con su respectivo número de calorías, gramos de proteína, gramos de grasa y gramos de carbohidratos.

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Lista de alimentos.txt to Lista de alimentos (1).txt


In [7]:
with open("Lista de alimentos.txt", "r") as file:
    lines = file.readlines()

print(lines[1:3])

['Leche descremada  en polvo,36,5.1,3.5,0.1)\n', 'Leche entera,57,4.5,3,3)\n']


Procesamiento de Datos:

Se eliminan los paréntesis al final de cada línea y se dividen los valores utilizando comas.

Se crean variables tipo float para los valores numéricos y se almacenan en una lista llamada alimentos_f.

In [8]:
alimentos_f = []
for line in lines:
  line = line.rstrip(")\n") #Quitar el paréntesis al final
  val = line.split(",") #dividir en comas
  if(len(val) == 5): #Quitamos de la lista los alimentos que tengan errores de indexación y convertimos los valores numéricos en variables tipo float
    val[1] = float(val[1])
    val[2] = float(val[2])
    val[3] = float(val[3])
    val[4] = float(val[4])
    alimentos_f.append(val)

#print(alimentos_f)

Selección Aleatoria de Alimentos:

    Se eligen aleatoriamente 15 alimentos de la lista procesada para formar la dieta inicial.


In [9]:
import random

alimentos = random.sample(alimentos_f,15)
print(alimentos[0])


['Queso fontina', 369.0, 0.0, 27.0, 29.0]



Definición de Requisitos Nutricionales:

    Se definen los requisitos nutricionales del individuo en términos de calorías, proteínas, grasas y carbohidratos.

In [10]:
#Definimos los requisitos
req = {
    "calorias" : float(3000),
    "proteinas" : float(70),
    "grasa" : float(85),
    "carbohidratos" : float(320),
}


Parámetros del Algoritmo Genético:

    Tamaño de la población (pop): 300 individuos.
    Número de generaciones (gens): 300 generaciones.
    Probabilidad de mutación (mut): 1%.

In [11]:

#Parametros del AG
pop = 300 #Población
gens = 300 #Generaciones
mut = 0.01 #Probabilidad de Mutación


In [12]:
print(req["calorias"])

3000.0


Función Objetivo (f_apt):

    Evalúa la aptitud de una dieta en función de su desviación con respecto a los requisitos nutricionales.
    Utiliza la suma ponderada de las diferencias en calorías, proteínas, grasas y carbohidratos.

In [13]:
def f_apt(dieta):
    calorias_dieta = sum(dieta[i] * alimentos[i][1] for i in range(len(dieta)))
    proteinas_dieta = sum(dieta[i] * alimentos[i][2] for i in range(len(dieta)))
    grasas_dieta = sum(dieta[i] * alimentos[i][3] for i in range(len(dieta)))
    carbohidratos_dieta = sum(dieta[i] * alimentos[i][4] for i in range(len(dieta)))

    # Calcular la puntuación de la dieta en función de la diferencia con los objetivos, normalizada para que los valores de una variable no opaquen los de otra
    puntuacion = abs(req["calorias"] - calorias_dieta)/req["calorias"] + abs(req["proteinas"] - proteinas_dieta)/req["proteinas"] + abs(req["grasa"] - grasas_dieta)/req["grasa"] + abs(req["carbohidratos"] - carbohidratos_dieta)/req["carbohidratos"]

    return puntuacion

Función para Crear Dieta Inicial (crear_dieta):

    Genera una dieta aleatoria inicialmente.

In [14]:
#Creamos la función que toma combinaciones de los 15 alimentos seleccionados al azar
def crear_dieta():
    return [random.randint(0, 10) for _ in alimentos]

Algoritmo Genético (alg_gen):

    Se crea una población inicial de dietas aleatorias.
    Durante un número especificado de generaciones, se evalúan y seleccionan las mejores dietas.
    Se generan nuevas dietas mediante cruces y mutaciones.
    La mejor dieta encontrada se devuelve junto con su puntuación de aptitud.

In [15]:
#Algoritmo genético
def alg_gen(num_g,pop_s):
  poblacion = [crear_dieta() for _ in range(pop_s)]

  for gen in range(num_g):
    poblacion = sorted(poblacion, key=f_apt) #Evaluar las dietas, poner las mejores al principio
    poblacion = poblacion[:pop_s//2] #Conservar la mejor mitad

    pob_n = [] #Generar la otra mitad mediante crucey mutación
    for _ in range(pop_s):
            padre = random.choice(poblacion)
            madre = random.choice(poblacion)
            punto_cruce = random.randint(1, len(alimentos) - 1)
            hijo = padre[:punto_cruce] + madre[punto_cruce:]

            # Aplicar mutación con una cierta probabilidad
            if random.random() < mut:
                indice_mutacion = random.randint(0, len(alimentos) - 1)
                hijo[indice_mutacion] = random.randint(0, 5)

            pob_n.append(hijo)
    poblacion = pob_n
  mejor_dieta = poblacion[0]
  return mejor_dieta, f_apt(mejor_dieta)

Ejecución y Resultados:

    El AG se ejecuta con los parámetros definidos.
    Se muestra la mejor dieta encontrada y su puntuación de aptitud.

In [16]:
mejor_dieta, puntuacion_mejor_dieta = alg_gen(gens,pop)
print("Mejor dieta encontrada:")
for i, cantidad in enumerate(mejor_dieta):
    print(f"{alimentos[i][0]}: {cantidad} unidades")

print(f"Puntuación de la dieta: {puntuacion_mejor_dieta}")

Mejor dieta encontrada:
Queso fontina: 0 unidades
Hígado de vaca: 0 unidades
Escarola: 1 unidades
Choclo: 1 unidades
Queso de cabra: 0 unidades
Queso tofu: 1 unidades
Maní: 0 unidades
Manteca: 2 unidades
Salchicha de viena: 4 unidades
Mondongo de vaca: 0 unidades
Yogur descr. saborizado: 4 unidades
Jamón cocido: 0 unidades
Ñoquis de papa: 0 unidades
Zapallito: 2 unidades
Cuadril magro: 0 unidades
Puntuación de la dieta: 0.19558315826330558
